In [ ]:
#Exercise 1

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import (
    RobertaForSequenceClassification,
    RobertaTokenizerFast,
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
    TrainingArguments,
    Trainer
)

In [ ]:
train_data = pd.read_csv('train_en.txt', sep='\t')
train_data.drop(columns=['User'], inplace=True)

In [ ]:

test_data = pd.read_csv('test_en.txt', sep='\t')
val_data = pd.read_csv('val_en.txt', sep='\t')


test_data.drop(columns=['User'], inplace=True)
val_data.drop(columns=['User'], inplace=True)

In [ ]:
positive = train_data[train_data['Style'] == 'positive']
negative = train_data[train_data['Style'] == 'negative']

positive_sample = positive.sample(15000, random_state=42)
negative_sample = negative.sample(15000, random_state=42)

selected_indexes = positive_sample.index.union(negative_sample.index)

balanced_train = pd.concat([positive_sample, negative_sample]).sample(frac=1, random_state=42)
balanced_train = balanced_train.reset_index(drop=True)
balanced_train.to_csv("balanced_train.csv", index=False)
remaining = train_data.drop(selected_indexes)


In [ ]:
pos = test_data[test_data['Style'] == 'positive']
neg = test_data[test_data['Style'] == 'negative']
print(len(pos), len(neg))

30853 12011


In [ ]:
positive = test_data[test_data['Style'] == 'positive']
negative = test_data[test_data['Style'] == 'negative']

positive_sample = positive.sample(2500, random_state=42)
negative_sample = negative.sample(2500, random_state=42)

balanced_test = pd.concat([positive_sample, negative_sample]).sample(frac=1, random_state=42)
balanced_test = balanced_test.reset_index(drop=True)

In [ ]:
balanced_test.head()

,Sentence,Style
0,I was looking for a cute coffee shop to come b...,positive
1,"The service is good, but the insensitive owner...",negative
2,"I thought with the trend in fro yo, you couldn...",negative
3,My first 10mile race...yahhh for me. I can't b...,positive
4,Great dine in theater experience. I purchased ...,positive


In [ ]:
pos = val_data[val_data['Style'] == 'positive']
neg = val_data[val_data['Style'] == 'negative']
print(len(pos), len(neg))

30853 12010


In [ ]:
positive = val_data[val_data['Style'] == 'positive']
negative = val_data[val_data['Style'] == 'negative']

positive_sample = positive.sample(500, random_state=42)
negative_sample = negative.sample(500, random_state=42)

balanced_val = pd.concat([positive_sample, negative_sample]).sample(frac=1, random_state=42)
balanced_val = balanced_val.reset_index(drop=True)

In [ ]:
class ToxicDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score

def compute_metrics(p):

    predictions = np.argmax(p.predictions, axis=1)


    accuracy = accuracy_score(p.label_ids, predictions)
    f1 = f1_score(p.label_ids, predictions, average='binary')
    precision=precision_score(p.label_ids,predictions,average='binary')
    recall=recall_score(p.label_ids,predictions,average='binary')

    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:
tokenizer=RobertaTokenizerFast.from_pretrained('roberta-base')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(
    balanced_train['Sentence'].tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    balanced_test['Sentence'].tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

val_encodings = tokenizer(
    balanced_val['Sentence'].tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

NameError: name 'tokenizer' is not defined

In [ ]:
train_labels = [1 if s == "negative" else 0 for s in balanced_train["Style"]]
test_labels = [1 if s == "negative" else 0 for s in balanced_test["Style"]]
val_labels = [1 if s == "negative" else 0 for s in balanced_val["Style"]]

train_dataset = ToxicDataset(train_encodings, train_labels)
test_dataset = ToxicDataset(test_encodings, test_labels)
val_dataset = ToxicDataset(val_encodings, val_labels)

NameError: name 'train_encodings' is not defined

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='toxic_roberta',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to='none'
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

Step,Training Loss
500,0.477800
1000,0.375700
1500,0.380400
2000,0.367400
2500,0.355000
3000,0.346200
3500,0.344600
4000,0.314200
4500,0.299200
5000,0.328300


TrainOutput(global_step=11250, training_loss=0.3012764397515191, metrics={'train_runtime': 3668.8557, 'train_samples_per_second': 24.531, 'train_steps_per_second': 3.066, 'total_flos': 5919998745600000.0, 'train_loss': 0.3012764397515191, 'epoch': 3.0})

In [ ]:
evaluation_results = trainer.evaluate(eval_dataset=test_dataset)
print(evaluation_results)

{'eval_loss': 0.5099393725395203, 'eval_accuracy': 0.8876, 'eval_f1_score': 0.8862348178137652, 'eval_precision': 0.8971311475409836, 'eval_recall': 0.8756, 'eval_runtime': 38.145, 'eval_samples_per_second': 131.079, 'eval_steps_per_second': 16.385, 'epoch': 3.0}


In [ ]:
len(test_encodings['input_ids'])


5000

In [ ]:
# Metrics from a trained RoBERTa for classification.

In [ ]:
# 'eval_loss': 0.5099393725395203,
#  'eval_accuracy': 0.8876,
#  'eval_f1_score': 0.8862348178137652,
#  'eval_precision': 0.8971311475409836,
#  'eval_recall': 0.8756,
#  'eval_runtime': 38.145,
#  'eval_samples_per_second': 131.079, 'eval_steps_per_second': 16.385, 'epoch': 3.0

In [ ]:
#Trying with higher learning rate

In [ ]:
training_args2 = TrainingArguments(
    output_dir='toxic_roberta',
    learning_rate=0.005,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to='none'
)


In [ ]:
model2 = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer2 = Trainer(
    model=model2,
    args=training_args2,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
trainer2.train()

Step,Training Loss
500,1.059300
1000,1.096300
1500,1.006000
2000,0.959000
2500,0.970200
3000,0.960300
3500,0.892000
4000,1.023500
4500,0.903800
5000,0.877800


TrainOutput(global_step=11250, training_loss=0.8920646375868055, metrics={'train_runtime': 3779.9956, 'train_samples_per_second': 23.81, 'train_steps_per_second': 2.976, 'total_flos': 5919998745600000.0, 'train_loss': 0.8920646375868055, 'epoch': 3.0})

In [ ]:
evaluation_results = trainer2.evaluate(eval_dataset=test_dataset)
print(evaluation_results)

{'eval_loss': 0.6941354274749756, 'eval_accuracy': 0.5, 'eval_f1_score': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 32.5795, 'eval_samples_per_second': 153.471, 'eval_steps_per_second': 19.184, 'epoch': 3.0}


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# 'eval_loss': 0.6939905881881714,
# 'eval_accuracy': 0.5,
# 'eval_f1_score': 0.0,
# 'eval_precision': 0.0,
# 'eval_recall': 0.0,
# 'eval_runtime': 6.5797,
# 'eval_samples_per_second': 151.983, 'eval_steps_per_second': 18.998, 'epoch': 3.0}


In [ ]:
# Just by making the learning rate higher we made the model unstable and
# got extremely bad results

In [ ]:
# DistilBERT

In [ ]:
from transformers import (
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast
)


In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(
    balanced_train['Sentence'].tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    balanced_test['Sentence'].tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

val_encodings = tokenizer(
    balanced_val['Sentence'].tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

In [ ]:
train_dataset = ToxicDataset(train_encodings, train_labels)
test_dataset = ToxicDataset(test_encodings, test_labels)
val_dataset=ToxicDataset(val_encodings,val_labels)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='toxic_distilbert',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to='none'
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

Step,Training Loss
500,0.713500
1000,0.712300
1500,0.720100
2000,0.697300
2500,0.698900
3000,0.700000
3500,0.698900
4000,0.697100
4500,0.695200
5000,0.699500


TrainOutput(global_step=11250, training_loss=0.6987845933702257, metrics={'train_runtime': 2431.9094, 'train_samples_per_second': 37.008, 'train_steps_per_second': 4.626, 'total_flos': 2980516469760000.0, 'train_loss': 0.6987845933702257, 'epoch': 3.0})

In [ ]:
evaluation_results = trainer.evaluate(eval_dataset=test_dataset)
print(evaluation_results)

{'eval_loss': 0.6931899785995483, 'eval_accuracy': 0.5, 'eval_f1_score': 0.6666666666666666, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_runtime': 16.795, 'eval_samples_per_second': 297.708, 'eval_steps_per_second': 37.213, 'epoch': 3.0}


In [ ]:
# 'eval_loss': 0.6931853294372559,
# 'eval_accuracy': 0.5,
# 'eval_f1_score': 0.6666666666666666,
#  'eval_precision': 0.5,
#  'eval_recall': 1.0,
#  'eval_runtime': 4.1374,
#  'eval_samples_per_second': 241.697, 'eval_steps_per_second': 30.212, 'epoch': 3.0

In [ ]:
#The model always outputs positive class

In [ ]:
#Trying DistilBERT with more epochs and lower learning rate

In [ ]:
training_args2 = TrainingArguments(
    output_dir='toxic_distilbert',
    learning_rate=1e-5,
    num_train_epochs=4,
    weight_decay=0.01,
    report_to='none'
)


In [ ]:
model2 = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer2 = Trainer(
    model=model2,
    args=training_args2,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
trainer2.train()

Step,Training Loss
500,0.454900
1000,0.389500
1500,0.383300
2000,0.351800
2500,0.340500
3000,0.353000
3500,0.350600
4000,0.327000
4500,0.296600
5000,0.301700


TrainOutput(global_step=15000, training_loss=0.2794489537556966, metrics={'train_runtime': 2640.8354, 'train_samples_per_second': 45.44, 'train_steps_per_second': 5.68, 'total_flos': 3974021959680000.0, 'train_loss': 0.2794489537556966, 'epoch': 4.0})

In [ ]:
evaluation_results = trainer2.evaluate(eval_dataset=test_dataset)
print(evaluation_results)

{'eval_loss': 0.5888851284980774, 'eval_accuracy': 0.8624, 'eval_f1_score': 0.860559383867045, 'eval_precision': 0.8722267871815941, 'eval_recall': 0.8492, 'eval_runtime': 18.7808, 'eval_samples_per_second': 266.229, 'eval_steps_per_second': 33.279, 'epoch': 4.0}


In [ ]:
#  'eval_loss': 0.5888851284980774,
#   'eval_accuracy': 0.8624,
#   'eval_f1_score': 0.860559383867045,
#   'eval_precision': 0.8722267871815941,
#   'eval_recall': 0.8492,
#   'eval_runtime': 18.7808, 'eval_samples_per_second': 266.229, 'eval_steps_per_second': 33.279, 'epoch': 4.0}

In [ ]:
# By lowering the learning rate and training in more epochs
# we can see that the model got a lot better.

In [ ]:
# Results from model from Lab1
# {'accuracy': 0.8620754012691303,
#  'precision': 0.7648284845853235,
#  'recall': 0.733244525851303,
#  'f1_score': 0.7487035620164924}

In [ ]:
# We can conclude that both RoBERTa and DistilBERT perform better than our RNN

In [ ]:
#Exercise 2

In [1]:
import pandas as pd
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [2]:
balanced_train=pd.read_csv('balanced_train.csv',sep=',')
balanced_train.head()


,Sentence,Style
0,So grateful I found this wonderful place! I've...,positive
1,"As far as government establishments go, this i...",negative
2,$10.75 for a glass of bad Pinot Grigio???\nSer...,negative
3,Horrible HORRIBLE place to live in. Laundry ar...,negative
4,"Yes, it's just a chain. But I am happy with th...",positive


In [ ]:
model_name="t5-base"

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def zero_shot_t5(sentences, labels, model, tokenizer, max_length=15):

    y_true = []
    y_pred = []

    for i, sentence in enumerate(sentences):
        y_true.append(labels[i])

        prompt = (
            "Sentiment analysis.\n"
            f"Text: {sentence}\n"
            "Classify the sentence as either POSITIVE or NEGATIVE"
        )

        tokens = tokenizer(prompt, return_tensors='pt', truncation=True)
        output_ids = model.generate(**tokens, max_length=max_length)
        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True).lower()

        print(prediction)

        if prediction in ['false', 'no', 'negative']:
            y_pred.append(1)
        else:
            y_pred.append(0)

    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred)
    }

    return {"predictions": y_pred, "metrics": metrics}


In [6]:
labels = [1 if s=='negative' else 0 for s in balanced_train['Style']]
sentences = balanced_train['Sentence'].tolist()

In [3]:


results = zero_shot_t5(sentences, labels, model, tokenizer)
print("Metrics:", results["metrics"])

NameError: name 'zero_shot_t5' is not defined

In [31]:
# NOTE: the code was executed on a separate machine with better specs because of
# google colab limits
# Metrics:
#  {'accuracy': 0.7697333333333334, 'precision': 0.7156716417910448,
#   'recall': 0.8950666666666667, 'f1': 0.7953791469194312}

In [9]:
model_name = "facebook/bart-large-mnli"

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

classifier = pipeline("zero-shot-classification", model=model_name)
def zero_shot_BART(sentences, labels):
    y_pred = []
    y_true = labels

    candidate_labels = ["positive", "negative"]

    for sent in sentences:
        result = classifier(sent, candidate_labels)
        pred_label = result["labels"][0]
        if pred_label == "negative":
            y_pred.append(1)
        else:
            y_pred.append(0)

    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred)
    }

    return {"predictions": y_pred, "metrics": metrics}


Device set to use cuda:0


In [16]:
results = zero_shot_BART(sentences[:1000], labels[:1000])
print("Metrics:", results["metrics"])

Metrics: {'accuracy': 0.815, 'precision': 0.8424657534246576, 'recall': 0.7608247422680412, 'f1': 0.7995666305525461}


In [17]:
# Metrics:
#  {'accuracy': 0.815,
#   'precision': 0.8424657534246576,
#   'recall': 0.7608247422680412,
#   'f1': 0.7995666305525461
#   }

In [23]:
test_data=pd.read_csv("test_en.txt",sep="\t")
test_data.drop(columns=['User'],inplace=True)

In [27]:
positive = test_data[test_data['Style'] == 'positive']
negative = test_data[test_data['Style'] == 'negative']

positive_sample = positive.sample(2, random_state=42)
negative_sample = negative.sample(2, random_state=42)

df=pd.concat([positive_sample,negative_sample]).sample(frac=1,random_state=42)
df=df.reset_index(drop=True)
df.head()



,Sentence,Style
0,Huh. What used to be known as the Peninsular L...,positive
1,This venue is very nice but dark! I do wish th...,negative
2,Pho-Nomenal!\n\nThis is an AWESOME mom & pop r...,positive
3,The only thing I like about RA sushi is their ...,negative


In [8]:
examples = [
    {"text": "This place is amazing!", "label": "positive"},
    {"text": "Food was terrible.", "label": "negative"},
    {"text": "The staff was very polite.", "label": "positive"},
]

In [29]:
model_name="t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [48]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def few_shot_t5(sentences, labels, model, tokenizer, examples,max_length=15):

    y_true = []
    y_pred = []

    examples_text = ""
    for ex in examples:
        examples_text += (
            f"Text: {ex['text']}\n"
            f"Sentiment: {ex['label']}\n\n"
        )


    for i, sentence in enumerate(sentences):
        y_true.append(labels[i])



        prompt = (
            "Sentiment analysis.\n\n"
            f"{examples_text}"
            f"Text: {sentence}\n"
            "Sentiment:"
        )

        tokens = tokenizer(prompt, return_tensors='pt', truncation=True)
        output_ids = model.generate(**tokens, max_length=max_length)
        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True).lower()


        if prediction in ['false', 'no', 'negative']:
            y_pred.append(1)
        else:
            y_pred.append(0)

    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred)
    }

    return {"predictions": y_pred, "metrics": metrics}


In [50]:
results = few_shot_t5(sentences[:1000], labels[:1000], model, tokenizer,examples)



print("Metrics:", results["metrics"])

Metrics: {'accuracy': 0.528, 'precision': 0.5074798619102416, 'recall': 0.9092783505154639, 'f1': 0.6514032496307238}


In [ ]:
# metrics:
#  {'accuracy': 0.528,
#   'precision': 0.5074798619102416,
#   'recall': 0.9092783505154639,
#   'f1': 0.6514032496307238
#   }

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def few_shot_BART(sentences, labels, examples, model, tokenizer, max_length=64):
    y_pred = []
    y_true = labels

    examples_text = ""
    for ex in examples:
        examples_text += (
            f"Text: {ex['text']}\n"
            f"Sentiment: {ex['label']}\n\n"
        )

    for sent in sentences:

        prompt = (
            "Classify the sentiment.\n\n"
            f"{examples_text}"
            f"Text: {sent}\n"
            "Sentiment:"
        )

        tokens = tokenizer(prompt, return_tensors="pt", truncation=True)
        output = model.generate(**tokens, max_length=max_length)

        prediction = tokenizer.decode(output[0], skip_special_tokens=True).lower()



        if "negative" in prediction:
            y_pred.append(1)
        else:
            y_pred.append(0)

    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
    }

    return {"predictions": y_pred, "metrics": metrics}


In [4]:
model_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [16]:
results = few_shot_BART(sentences[:100], labels[:100],examples, model, tokenizer)



print("Metrics:", results["metrics"])

Metrics: {'accuracy': 0.56, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Metrics: {'accuracy': 0.56, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}

In this exercise the few-shot prompted models perform worse than the zero-shot models, especially the few-shot BART, which is worst overall. These models do not beat the models in exercise 1 nor the RNN from lab1 (Though they perfrom somewhat similarly)
.